# Comparison of RTM wrapper wavelength sweep with Py6S wavelength sweep.

In [1]:
import dataclasses

import alive_progress
import numpy as np
import numpy.testing
import Py6S

from rtm_wrapper.engines.sixs import PySixSEngine, make_sixs_wrapper, pysixs_default_inputs
from rtm_wrapper.execution import ConcurrentExecutor, SerialExecutor
from rtm_wrapper.simulation import SweepSimulation

Define sweep wavelengths.

In [2]:
wl_spacing = 0.0025
wl = np.arange(0.4, 2.5 + wl_spacing, wl_spacing)

Run wavelength sweep using RTM wrapper interface.

In [3]:
%%time
sweep = SweepSimulation(
    # Sweep wavelengths.
    {
        "wavelength": wl,
    },
    # Use default Py6S settings, but with tweaked aerosol profile.
    base=dataclasses.replace(
        pysixs_default_inputs(), 
        aerosol_profile="Continental",
        aerosol_aot=[(5, 1)],
    )
)

engine = PySixSEngine()
# runner = SerialExecutor()
runner = ConcurrentExecutor(max_workers=None)

with alive_progress.alive_bar(sweep.sweep_size, force_tty=True) as bar:
    runner.run(sweep, engine, lambda _: bar())

outputs_a = runner.collect_results()

|████████████████████████████████████████| 841/841 [100%] in 19.8s (41.23/s)    
CPU times: user 6.06 s, sys: 6.29 s, total: 12.3 s
Wall time: 20 s


Run wavelength sweep using Py6S interface.

In [4]:
%%time
sixs = make_sixs_wrapper()
sixs.aero_profile = Py6S.AeroProfile.UserProfile(Py6S.AeroProfile.Continental)
sixs.aero_profile.add_layer(5, 1)
_wl, outputs_b = Py6S.SixSHelpers.Wavelengths.run_wavelengths(sixs, wl)

Running for many wavelengths - this may take a long time
CPU times: user 4.94 s, sys: 5.39 s, total: 10.3 s
Wall time: 20.9 s


Verify that the results are the same.

In [5]:
np.testing.assert_allclose(
    outputs_a.variables["apparent_radiance"], [o.apparent_radiance for o in outputs_b]
)